In [1]:
import pandas as pd 
import numpy as np 
from funcs import *

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

data=pd.read_csv('testLL.csv')

ModuleNotFoundError: No module named 'funcs'